# Assignment 2
## Group name: ID2214-12
### Project members: 
[Ruben van Gemeren, rubenvg@kth.se]

[Johan Edeland, johede@kth.se]

[Sam Horácek, horacek@kth.se]

### Declaration:
By submitting this assignment, it is hereby declared that all group members listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214), no part of the solution has been provided by someone not listed as a project member above, and no part of the solution has been generated by a system.

It is furthermore declared that the submitted assignment will not be shared during the course, with any individual other than the group members listed above and teachers of the course ID2214/FID3214. In particular, the assignment will not be uploaded to any public repository. The submitted assignment can be shared after the course only if written consent has been provided by the course responsible of ID2214/FID3214.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy and pandas may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas and time

In [4]:
import numpy as np
import pandas as pd
import time

In [5]:
from platform import python_version

print(f"Python version: {python_version()}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

Python version: 3.11.5
NumPy version: 1.24.3
Pandas version: 2.2.1


## Reused functions from Assignment 1

In [3]:
# Copy and paste functions from Assignment 1 here that you need for this assignment
def accuracy(df, correctlabels):
    correct = 0

    for i, row in df.iterrows():
        predicted_label = row.idxmax()
        if predicted_label == correctlabels[i]:
            correct += 1

    return correct / len(correctlabels)


def brier_score(df, correctlabels):
    brier_score = 0

    for i, row in df.iterrows():
        correct_index = np.where(df.columns == correctlabels[i])[0][0]
        correct_vector = np.zeros(len(row))
        correct_vector[correct_index] = 1
        brier_score += np.sum((row - correct_vector) ** 2)

    return brier_score / len(correctlabels)

from typing import Counter


def auc(df, correctlabels):
    # Class labels and frequencies
    class_labels = df.columns
    n = len(correctlabels)
    class_counts = Counter(correctlabels)
    # Create a dict with key: c and value: frequency of c
    class_freq = {c: class_counts[c] / n for c in class_labels}

    print (class_freq)
    print (class_labels)
    print (class_counts)
    auc_total = 0

    # Calculate AUC for each class
    for c in class_labels:
        scores = df[c]
        is_class = [1 if label == c else 0 for label in correctlabels]

        print (is_class)

        # Create DataFrame for scores and correct labels, then sort by scores
        score_df = pd.DataFrame({'score': scores, 'is_class': is_class})
        score_df = score_df.sort_values(by='score', ascending=False)

        # Initialize variables
        tp, fp = 0, 0
        auc_class = 0
        tpr_prev, fpr_prev = 0, 0

        # Get the total counts for the class and non-class
        pos_total = class_counts[c]
        neg_total = n - pos_total

        # Calculate TPR and FPR for each score threshold
        for _, row in score_df.iterrows():
            if row['is_class'] == 1:
                tp += 1
            else:
                fp += 1

            tpr = tp / pos_total if pos_total > 0 else 0
            fpr = fp / neg_total if neg_total > 0 else 0

            # Accumulate AUC using trapezoidal rule
            auc_class += (fpr - fpr_prev) * (tpr + tpr_prev) / 2
            tpr_prev, fpr_prev = tpr, fpr

        # Weighted AUC by class frequency
        auc_total += class_freq[c] * auc_class

    return auc_total

def create_column_filter(df):
    df_copy = df.copy()
    column_filter = []

    for column in df_copy.columns:
        if column == "CLASS" or column == "ID":
            column_filter.append(column)
        else:
            unique_values = df_copy[column].dropna().unique()
            if len(unique_values) > 1:
                column_filter.append(column)
            else:
                df_copy.drop(column, axis=1, inplace=True)

    return df_copy, column_filter

def create_imputation(df):
    df_copy = df.copy()
    imputation = {}

    for column in df_copy.columns:
        if column == "CLASS" or column == "ID":
            continue
        if df_copy[column].dtype == "float" or df_copy[column].dtype == "int":
            mean = df_copy[column].mean()
            df_copy[column].fillna(mean, inplace=True)
            imputation[column] = mean
        elif df_copy[column].dtype == "object" or df_copy[column].dtype.name == "category":
            mode = df_copy[column].mode()[0]
            df_copy[column].fillna(mode, inplace=True)
            imputation[column] = mode

    return df_copy, imputation

def apply_imputation(df, imputation):
    df_copy = df.copy()

    for column in df_copy.columns:
        if column in imputation:
            df_copy[column].fillna(imputation[column], inplace=True)

    return df_copy

def create_normalization(df, normalizationtype="minmax"):
    df_copy = df.copy()
    normalization = {}

    for column in df_copy.columns:
        # If the columns are "CLASS" or "ID", leave them unchanged
        if column == "CLASS" or column == "ID":
            continue
        if df_copy[column].dtype == "float" or df_copy[column].dtype == "int":
            if normalizationtype == "minmax":
                min_value = df_copy[column].min()
                max_value = df_copy[column].max()
                df_copy[column] = (df_copy[column] - min_value) / (max_value - min_value)
                normalization[column] = ("minmax", min_value, max_value)
            elif normalizationtype == "zscore":
                mean = df_copy[column].mean()
                std = df_copy[column].std()
                df_copy[column] = (df_copy[column] - mean) / std
                normalization[column] = ("zscore", mean, std)

    return df_copy, normalization

def apply_normalization(df, normalization):
    df_copy = df.copy()

    for column in df_copy.columns:
        if column in normalization:
            normalizationtype, value1, value2 = normalization[column]
            if normalizationtype == "minmax":
                df_copy[column] = (df_copy[column] - value1) / (value2 - value1)
            elif normalizationtype == "zscore":
                df_copy[column] = (df_copy[column] - value1) / value2

    return df_copy

def create_one_hot(df):
    df_copy = df.copy()
    one_hot = {}

    for column in df_copy.columns:
        if column == "CLASS" or column == "ID":
            continue
        if df_copy[column].dtype == "object" or df_copy[column].dtype.name == "category":
            categories = df_copy[column].unique()
            for category in categories:
                new_column_name = column + "_" + str(category)
                df_copy[new_column_name] = (df_copy[column] == category).astype("float")
            df_copy.drop(column, axis=1, inplace=True)
            one_hot[column] = categories

    return df_copy, one_hot

def apply_one_hot(df, one_hot):
    df_copy = df.copy()

    for column in df_copy.columns:
        if column in one_hot:
            categories = one_hot[column]
            for category in categories:
                new_column_name = column + "_" + str(category)
                df_copy[new_column_name] = (df_copy[column] == category).astype("float")
            df_copy.drop(column, axis=1, inplace=True)

    return df_copy


## 1. Define the class kNN

In [7]:
# Define the class kNN with three functions __init__, fit and predict (after the comments):
#
# Input to __init__:
# self - the object itself
#
# Output from __init__:
# <nothing>
#
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, normalization, one_hot, labels, training_labels, training_data, training_time
#
# Input to fit:
# self              - the object itself
# df                - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# normalizationtype - "minmax" (default) or "zscore"
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter   - a column filter (see Assignment 1) from df
# self.imputation      - an imputation mapping (see Assignment 1) from df
# self.normalization   - a normalization mapping (see Assignment 1), using normalizationtype from the imputed df
# self.one_hot         - a one-hot mapping (see Assignment 1)
# self.training_labels - a pandas series corresponding to the "CLASS" column, set to be of type "category"
# self.labels          - a list of the categories (class labels) of the previous series
# self.training_data   - the values (an ndarray) of the transformed dataframe, i.e., after employing imputation,
#                        normalization, and possibly one-hot encoding, and also after removing the "CLASS" and "ID" columns
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# k    - an integer >= 1 (default = 5)
#
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the relative class frequencies in the set of class labels from the k nearest
#               (with respect to Euclidean distance) neighbors in training_data
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply column filtering, imputation, normalization and one-hot
#
# Hint 2: Get the numerical values (as an ndarray) from the resulting dataframe and iterate over the rows
#         calling some sub-function, e.g., get_nearest_neighbor_predictions(x_test,k), which for a test row
#         (numerical input feature values) finds the k nearest neighbors and calculate the class probabilities.
#
# Hint 3: This sub-function may first find the distances to all training instances, e.g., pairs consisting of
#         training instance index and distance, and then sort them according to distance, and then (using the indexes
#         of the k closest instances) find the corresponding labels and calculate the relative class frequencies

from scipy.spatial import distance

class kNN:
    def __init__(self):
        # Initialize attributes
        self.column_filter = None
        self.imputation = None
        self.normalization = None
        self.one_hot = None
        self.labels = None
        self.training_labels = None
        self.training_data = None
        self.training_time = None

    def fit(self, df, normalizationtype="minmax"):
        # Remove "CLASS" and "ID" columns
        df = df.drop(columns=["CLASS", "ID"], errors="ignore")

        # Column filter (keeping relevant columns)
        self.column_filter = create_column_filter

        # Imputation - fill missing values (for simplicity, we use mean imputation)
        self.imputation = create_imputation

        # Normalization
        self.normalization = create_normalization

        # One-hot encoding for categorical features
        self.one_hot = create_one_hot

        # Save the training labels and class categories
        self.training_labels = df["CLASS"].astype("category")
        self.labels = list(self.training_labels.cat.categories)

        # Store the processed training data as a NumPy array
        self.training_data = self.one_hot.values

    def predict(self, df, k=5):
        # Drop any "CLASS" and "ID" columns in test data
        df = df.drop(columns=["CLASS", "ID"], errors="ignore")

        # Apply column filter
        df = df[self.column_filter]

        # Impute missing values in test data using the same strategy as training
        df = apply_imputation(df, self.imputation)

        # Normalize test data using the same approach as in fit()
        df = apply_normalization(df, self.normalization)

        # One-hot encode the test data to match training data structure
        df = apply_one_hot(df, self.one_hot)

        # Convert the test data to NumPy array for distance calculations
        test_data = df.values
        predictions = []

        for x_test in test_data:
            prediction = self.get_nearest_neighbor_predictions(x_test, k)
            predictions.append(prediction)

        # Convert predictions to DataFrame with class labels as columns
        return pd.DataFrame(predictions, columns=self.labels)

    def get_nearest_neighbor_predictions(self, x_test, k):
        # Calculate distances between x_test and each training sample
        distances = [(idx, distance.euclidean(x_test, x_train)) for idx, x_train in enumerate(self.training_data)]

        # Sort by distance and select the k nearest neighbors
        k_nearest = sorted(distances, key=lambda x: x[1])[:k]

        # Get the classes of the k nearest neighbors
        k_labels = [self.training_labels.iloc[idx] for idx, _ in k_nearest]

        # Calculate class probabilities
        class_counts = pd.Series(k_labels).value_counts(normalize=True)
        return [class_counts.get(label, 0) for label in self.labels]



In [8]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

knn_model = kNN()

t0 = time.perf_counter()
knn_model.fit(glass_train_df)
print("Training time: {0:.2f} s.".format(time.perf_counter()-t0))

test_labels = glass_test_df["CLASS"]

k_values = [1,3,5,7,9]
results = np.empty((len(k_values),3))

for i in range(len(k_values)):
    t0 = time.perf_counter()
    predictions = knn_model.predict(glass_test_df,k=k_values[i])
    print("Testing time (k={0}): {1:.2f} s.".format(k_values[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=k_values,columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

KeyError: 'CLASS'

In [5]:
train_labels = glass_train_df["CLASS"]
predictions = knn_model.predict(glass_train_df,k=1)
print("Accuracy on training set (k=1): {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set (k=1): {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set (k=1): {0:.4f}".format(brier_score(predictions,train_labels)))

Accuracy on training set (k=1): 1.0000
AUC on training set (k=1): 1.0000
Brier score on training set (k=1): 0.0000


### Comment on assumptions, things that do not work properly, etc.


## 2. Define the class NaiveBayes

In [ ]:
# Define the class NaiveBayes with three functions __init__, fit and predict (after the comments):
#
# Input to __init__:
# self - the object itself
#
# Output from __init__:
# <nothing>
#
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, binning, labels, class_priors, feature_class_value_counts, feature_class_counts
#
# Input to fit:
# self    - the object itself
# df      - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins  - no. of bins (default = 10)
# bintype - either "equal-width" (default) or "equal-size"
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter              - a column filter (see Assignment 1) from df
# self.binning                    - a discretization mapping (see Assignment 1) from df
# self.class_priors               - a mapping (dictionary) from the labels (categories) of the "CLASS" column of df,
#                                   to the relative frequencies of the labels
# self.labels                     - a list of the categories (class labels) of the "CLASS" column of df
# self.feature_class_value_counts - a mapping from the feature (column name) to the number of
#                                   training instances with a specific combination of (non-missing, categorical)
#                                   value for the feature and class label
# self.feature_class_counts       - a mapping from the feature (column name) to the number of
#                                   training instances with a specific class label and some (non-missing, categorical)
#                                   value for the feature
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
#
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the naive approximation of Bayes rule (see lecture slides)
#
# Hint 1: First apply the column filter and discretization
#
# Hint 2: Iterating over either columns or rows, and for each possible class label, calculate the relative
#         frequency of the observed feature value given the class (using feature_class_value_counts and
#         feature_class_counts)
#
# Hint 3: Calculate the non-normalized estimated class probabilities by multiplying the class priors to the
#         product of the relative frequencies
#
# Hint 4: Normalize the probabilities by dividing by the sum of the non-normalized probabilities; in case
#         this sum is zero, then set the probabilities to the class priors
#
# Hint 5: To clarify the assignment text a little: self.feature_class_value_counts should be a mapping from
#         a column name (a specific feature) to another mapping, which given a class label and a value for
#         the feature, returns the number of training instances which have included this combination,
#         i.e., the number of training instances with both the specific class label and this value on the feature.
#
# Hint 6: As an additional hint, you may take a look at the slides from the NumPy and pandas lecture, to see how you
#         may use "groupby" in combination with "size" to get the counts for combinations of values from two columns.




In [7]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

nb_model = NaiveBayes()

test_labels = glass_test_df["CLASS"]

nobins_values = [3,5,10]
bintype_values = ["equal-width","equal-size"]
parameters = [(nobins,bintype) for nobins in nobins_values for bintype in bintype_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    nb_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = nb_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values]),
                       columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

Training time (3, 'equal-width'): 0.10 s.
Testing time (3, 'equal-width'): 0.09 s.
Training time (3, 'equal-size'): 0.09 s.
Testing time (3, 'equal-size'): 0.07 s.
Training time (5, 'equal-width'): 0.10 s.
Testing time (5, 'equal-width'): 0.08 s.
Training time (5, 'equal-size'): 0.08 s.
Testing time (5, 'equal-size'): 0.07 s.
Training time (10, 'equal-width'): 0.09 s.
Testing time (10, 'equal-width'): 0.07 s.
Training time (10, 'equal-size'): 0.10 s.
Testing time (10, 'equal-size'): 0.07 s.



'results'

Accuracy  Brier score       AUC
3  equal-width  0.616822     0.622116  0.724335
   equal-size   0.607477     0.554782  0.780163
5  equal-width  0.644860     0.551101  0.771688
   equal-size   0.598131     0.581556  0.796675
10 equal-width  0.654206     0.527569  0.812887
   equal-size   0.588785     0.741668  0.751165

In [8]:
train_labels = glass_train_df["CLASS"]
nb_model.fit(glass_train_df)
predictions = nb_model.predict(glass_train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.8505
AUC on training set: 0.9687
Brier score on training set: 0.2263


### Comment on assumptions, things that do not work properly, etc.